In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

'Why did the ice cream truck break down?\n\nBecause it had a rocky road!'

In [2]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [4]:
prompt_value.to_messages()
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

In [5]:
message = model.invoke(prompt_value)
message

AIMessage(content='Why did the ice cream truck break down?\n\nBecause it had too many sundaes!', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 15, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [6]:
%pip install --upgrade --quiet  langchain langchain-openai duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [28]:
from langchain.tools import DuckDuckGoSearchRun
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

search = DuckDuckGoSearchRun()

In [2]:
template = """turn the following user input into a search query for a search engine:

{input}"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [3]:
chain = prompt | model | StrOutputParser() | search

In [4]:
chain.invoke({"input": "I'd like to figure out what games are tonight"})

'Sports Games Today has established itself as a leading and trusted online platform for up-to-date live sports TV schedules, ensuring that fans are well-informed and engaged with their favorite sports. Our comprehensive schedules for today cover a wide array of sports, including football, basketball, baseball, hockey, motorsports, soccer, and more. How to live stream NBA games tonight. Pelicans vs Thunder and Clippers vs Lakers will air on ESPN. Viewers can also stream NBA games on Sling TV. Fans in the U.S. can watch the biggest games of ... What Channel and Time do NHL Games Play on TV Today, Tonight, Tomorrow? Watch and stream NHL live games today on TV. 2023-24 NHL TV Schedule for tonight includes games on ABC, ESPN, TNT, TBS & Hulu. NHL on TV Wednesday, 3/27/2024. NHL REGULAR SEASON TIME ET TV; Ottawa at Buffalo: 7:30pm: Listings for all NFL Network programs -Good Morning Football, NFL Total Access, Thursday Night Football & more. The official source for NFL news, video highlights

In [60]:
import requests
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.tools.render import render_text_description

class LockColor(BaseModel):
    color: str = Field(description="first color")
    lock_type: str = Field(description="The type of lock to be updated. Can be either 'open' or 'closed' or 'none' or 'half-opened'")
    message: str= Field(description="The message to be sent back to the user. Either 'success' or 'failure'")

@tool
def sendRequest(color: str):
    """Send a request to localhost"""
    # response = requests.post("http://localhost:3000/api/lockcolor", json={"color": color})
    return {"color": color}

req = StructuredTool.from_function(
    func=sendRequest,
    name="SendRequest",
    description="Send a request to localhost",
    args_schema=LockColor
)

In [61]:
sendRequest.invoke('blue')

{'color': 'blue'}

In [62]:
rendered_tools = render_text_description([sendRequest])
rendered_tools

'sendRequest: sendRequest(color: str) - Send a request to localhost'

In [63]:
model = ChatOpenAI(model="gpt-3.5-turbo")


# prompt = ChatPromptTemplate.from_template("send a request to localhost with the color {color}. Return the response of the function and NOTHING ELSE.")

system_prompt = f"""
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)


# output_parser = StrOutputParser()
output_parser = JsonOutputParser()

chain = prompt | model | output_parser

In [64]:
chain.invoke({"input": "blue"})

{'name': 'sendRequest', 'arguments': {'color': 'blue'}}

In [66]:
prompt.invoke({"input": "blue"})

ChatPromptValue(messages=[SystemMessage(content="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nsendRequest: sendRequest(color: str) - Send a request to localhost\n\nGiven the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.\n"), HumanMessage(content='blue')])